In [ ]:
# Crypto heatmap

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=10&e=Coinbase"
f = requests.get(url)
ipdata = f.json()
pd.DataFrame(ipdata['Data']).head(5)

In [ ]:
def get_data(date):
    """query API for 2000 days historical price data starting from "date"."""
    url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=2000&toTs={}".format(
        date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [ ]:
def get_df(from_date, to_date):
    date = to_date
    holder = []
    # while earliest date returned is later than the earliest data requested, keep querying API # and adding results to list
    while date > from_date:
        data = get_data(date)
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom']
        # join together all api queries from list
        df = pd.concat(holder, axis=0)
        # remove data points from before from_date
        df = df[df['time'] > from_date]
        # convert to timestamp
        df['time'] = pd.to_datetime(df['time'], unit='s')
        # make datafrom index the time
        df.set_index('time', inplace=True)
        # sort in time order
        df.sort_index(ascending=False, inplace=True)
        return df

In [ ]:
df = get_df(1569801600, 1572029538)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.plot(df[['low', 'close', 'high']])
ax.set_ylabel('BTC Price (USD)')
ax.set_xlabel('Date')
ax.legend(['Low', 'Close', 'High'])

In [ ]:
def get_data_spec(coin, date, time_period):
    """query api for historical data starting from "date"."""
    url = "https://min-api.cryptocompare.com/data/{}?fsym={}&tsym=USD&Limit=2000&toTs={}".format(
        time_period, coin, date)
    r = requests.get(url)
    ipdata = r.json()
    return ipdata

In [ ]:
def get_df_spec(time_period, coin, from_date, to_date):
    """ get historical price data between dates. If further apart query multiple times."""
    date = to_date
    holder = []
    while date > from_date:
        data = get_data_spec(coin, date, time_period)
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom']
    df = pd.concat(holder, axis=0)
    df = df[df['time'] > from_date]
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.sort_index(ascending=False, inplace=True)
    df.rename(columns={'close': coin}, inplace=True)
    return df[coin]

In [ ]:
coins = ['BTC', 'ETH', 'LTC', 'DOGE']

In [ ]:
holder = []
from_date = 1569801600  # 09-30-2019
to_date = 1572029538  # Today
time_period = 'histoday'
for coin in coins:
    holder.append(get_df_spec(time_period, coin, from_date, to_date))
df = pd.concat(holder, axis=1)

In [ ]:
# convert each column of df to be rate of return instead of price
df = df.divide(df.shift()) - 1
df_norm_col = (df - df.mean()) / df.std()

In [ ]:
import seaborn as sns
plt.figure(figsize=(15, 10))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(
    df.corr(),
    annot=True,
    annot_kws={"size": 10},
    linewidth=0.5,
    cmap=cmap)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15, 10))

sns.pairplot(df, kind='reg', palette='Set2')
plt.show()